<a href="https://colab.research.google.com/github/Junhojuno/keras-tutorial/blob/master/07_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 함수형 API
- 기존 Sequential과 함수형 API 비교
- compile부분부터는 기존 Sequential과 동일

In [2]:
from keras.models import Sequential, Model
from keras.layers import *
from keras import Input

# 기존 Sequential 모델
seq_model = Sequential()
seq_model.add(Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(Dense(32, activation='relu'))
seq_model.add(Dense(10, activation='softmax'))

seq_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
# 함수형 API
input_tensor = Input(shape=(64,))
x = Dense(32, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)
output_tensor = Dense(10, activation='softmax')(x)

functional_model = Model(input_tensor, output_tensor)

functional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### 다중 입력 모델
- 함수 API는 다중입력모델을 만드는데 사용할 수 있다.
- 가장 간단한 Question-Answering 모델 구현
  - 입력으로 정보가 담긴 text와 질문이 들어감
  - 답을 출력하도록 하는 모델 구성

In [7]:
from keras.models import Model
from keras.layers import *
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(input_dim=text_vocabulary_size, output_dim=64)(text_input)
encoded_text = LSTM(32)(embedded_text) # (sample,features)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(input_dim=question_vocabulary_size, output_dim=32)(question_input)
encoded_question = LSTM(16)(embedded_question) # (sample,features)

# encoding된 질문과 텍스트를 연결
concatenated = concatenate([encoded_text, encoded_question], axis=-1)

answer = Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS